In [1]:
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *

# qiskit-ibmq-provider has been deprecated.
# Please see the Migration Guides in https://ibm.biz/provider_migration_guide for more detail.
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Estimator, Session, Options

# Loading your IBM Quantum account(s)
service = QiskitRuntimeService(channel="ibm_quantum")

# Invoke a primitive. For more details see https://qiskit.org/documentation/partners/qiskit_ibm_runtime/tutorials.html
# result = Sampler("ibmq_qasm_simulator").run(circuits).result()

In [ ]:
#Imports
import numpy as np
from qiskit.tools.jupyter import *
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
from qiskit_nature import settings
settings.use_pauli_sum_op = False
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.algorithms.optimizers import SLSQP
from qiskit.primitives import Estimator
from qiskit_nature.second_q.circuit.library import HartreeFock
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit.primitives import Estimator
from qiskit.circuit.library import EfficientSU2

def get_driver(dist):
    # Define driver for BeH2 Molecule
    driver = PySCFDriver(
        atom=f"Si 0 0 0; Ge 0 0 {dist}",
        basis="sto3g",
        charge=0,
        spin=0,
        unit=DistanceUnit.ANGSTROM,
    )
    return driver

def transform(problem):
    # specify active space transformation
    active_space_trafo = ActiveSpaceTransformer(
    num_electrons=problem.num_particles, num_spatial_orbitals=5  # Updated to appropriate number of spatial orbitals for fluorine
    )

    # Transform the electronic structure problem
    problem = active_space_trafo.transform(problem)
    return problem

def get_ansatz(problem, mapper):
    q_op = mapper.map(problem.second_q_ops()[0])
    num_qubits = q_op.num_qubits
    ansatz = EfficientSU2(
        num_qubits = num_qubits, 
        reps = 1, 
        entanglement = "linear", 
        insert_barriers=True
    )

    return ansatz
def vqe_solver(problem,q_op,mapper):
    # Define the VQE solver
    vqe = VQE(
        estimator = Estimator(), 
        ansatz = get_ansatz(problem,mapper), 
        optimizer = SLSQP()
    )
    groundstate_solver = GroundStateEigensolver(mapper, vqe)
    result = groundstate_solver.solve(problem)
    return result

distances = np.arange(0.3, 4.0, 0.1)

vqe_energy = []
if __name__ == '__main__':
   for dist in distances:

    # Define the problem using driver
    problem = get_driver(dist).run()

    # apply transformation
    problem = transform(problem)
   
    mapper = ParityMapper(num_particles=problem.num_particles)
    q_op = mapper.map(problem.second_q_ops()[0])
    vqe_result = vqe_solver(problem, q_op,mapper)
    vqe_energy.append(vqe_result.total_energies[0].real)
    print(f"Interatomic Distance: {np.round(dist, 2)}", f"VQE Result: {vqe_energy[-1]:.5f}")

Interatomic Distance: 0.3 VQE Result: -2050.52662


/opt/conda/lib/python3.10/site-packages/scipy/optimize/_optimize.py:284: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "


Interatomic Distance: 0.4 VQE Result: -2267.13827
Interatomic Distance: 0.5 VQE Result: -2300.61866
Interatomic Distance: 0.6 VQE Result: -2089.40597
Interatomic Distance: 0.7 VQE Result: -2326.58609
Interatomic Distance: 0.8 VQE Result: -2331.20135
Interatomic Distance: 0.9 VQE Result: -2333.58286
Interatomic Distance: 1.0 VQE Result: -2334.90513
